# Python for Librarians - Week 4 Workalong

For this week's material we are going to really dig into our theme of data science. We are going to be doing some machine learning on the same data set we saw in week 2. The point of this will be to determine if we can derive any **insights** from the data. Basically any characterists, themes, trends, or patterns we might be able to make some predictions with. The idea is that these **insights** would be difficult for a person to spot by looking at the data directly. We are going to rely on the computer to do this for us. In this case we'll be using the [SciKit Learn](https://scikit-learn.org/stable/) Python Library to do this.

We will also build more experience with visualizations using the matplotlib Python Library.

Please make sure you watch the Week 4 introductory video for a brief lecture on how Machine Learning Works. It will help fill in details presented in this notebook.

# For your notes!

This notebook (and accompanying answers) are for you to work through to learn the material. You do not need to submit this via Colab or via the Library Juice Press Site.

If you are having any problems with questions in this notebook please post to the **Week 4 Help** forum.

## Machine Learning Basics

You'll see lots of news stories these days about algorithms, how they are applied, and the way they are exploited. Certainly the readings you've been examining for this class have been build around this theme. Our investigation into machine learning (ML for short) is meant to pull back the curtain around the mystique. At the end of the material you'll have a good understanding of the weaknesses of ML. It's built on automatically making guesses on data that it is fed. Yup, that's about it. ML practitioners don't often brag about this.

So with that said don't let the impressive name fool you. ML is basically easily defined in the list of following steps:

1. Getting your data and cleaning it up
1. Identify what parts of your data are **features**
1. Identify what is your **target variable** or the thing you want to guess based on the features
1. Split your data in **training and testing sets**
1. **Train** your model against the training dataset
1. **Test** your model against the testing dataset
1. ????
1. Profit

Or in other words, we take a big pile of data, pick one thing in that data we'd like to minimize/maximize and we use the relations to the other columns of data to make that happen. This will get more clear as we continue. 

We are going to be doing a [classification](https://en.wikipedia.org/wiki/Statistical_classification) problem. The diagram that follows is a good representation of this:

![classification](https://raw.githubusercontent.com/BrockDSL/Machine_Learning_with_Python/master/classification.png)

So basically we have a collection of data points (some blue, some red) and we want to divide those points into two classifications using some automatic process. The right side of the diagram shows one potential classification. We have 'class' of blue points, another of red points, and another class for those in the middle that we can't reliable group into blue or red.


## Case Study: Decision Trees & Titanic Survivors

A decision tree is one of the most basic machine learning model you can use. It is considered a [supervised learning](https://en.wikipedia.org/wiki/Supervised_learning) method. You create the best [decision tree](https://en.wikipedia.org/wiki/Decision_tree_learning) that you can based on your training data. Here's an example tree that shows your chance of surviving the Titanic disaster. What we are creating is series of question that when answered will put observations into a _bucket_ or in other words one of the classification options. We also devise a probability associated with an observation falling into that _bucket_.

The features are described by the labels, however ``sibsp`` - is the number of spouses or siblings on board.

![dtree](https://upload.wikimedia.org/wikipedia/commons/e/eb/Decision_Tree.jpg)

You start at the root of the tree and answer the questions about your data point. Eventually you'll hit a bucket that you can put your data point in. In our example here we have two different buckets: **survived** & **died**. Put in another way, this tree tells us that the most important question to ask first is what is the gender of the person you are considering, then next most important question is their age above 9 and a half, followed lastly by, does this person have less than three spouses or siblings on board. 

With all of that preamble out of the way. Let's start some code. Run the next cell to load up all of the Libraries we'll need for this week's lesson.

In [ ]:
#This should look familar
import pandas as pd
import numpy

#We'll draw a graph later on
import matplotlib.pyplot as plt

#Our 'ML pieces'
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_text


#Suppress the distracting warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

This next two cells should look familar, we are loading up the CSV file from week two and putting it in a pandas dataframe. (The dataset is a bit modified and truncated just to make life a bit easier)

In [ ]:
#We'll reuse the same data set from Week 2, the circulation stats from
# the SF Library system
sf_data = pd.read_csv("https://raw.githubusercontent.com/elibtronic/lja_datasets/master/week_4_workalong_san_francisco.csv")


In [ ]:
#Run this a few times to seem more glimpses of the data.
sf_data.sample(10)

In [ ]:
#Let's apply our describe function to this dataframe
sf_data.describe()

----

## Prepping the data for our ML model

Our **Target** variable is going to be based on the following:

In [ ]:
#Let's use our Panda knowledge to see what categories are we looking at
sf_data["Patron Type Definition"].value_counts()

### Our Target

So we have 3 types of patrons:
- category `1` which represents the **'Adults'** & **'Seniors'** from the original dataset 
- category `2` which represents the **'Young Adult'** users from the original dataset
- category `0` which represents the misc. categories of users from the original dataset

This will be our 'target' or the thing we'll try to guess. Why are these values numbers instead of text labels? Turns out the Python code to construct these models requires all our values to be numbers. If you're interested in learning more about the process of modifying your data in this way it is called [one-hot encoding](https://en.wikipedia.org/wiki/One-hot)

### Our Features

We are going to build our ML model to be able to guess what category a patron is based on:
- total checkouts
- total renewals
- circulation active year

These 3 columns will be our 'features' that will help formulate our gueses.


### The ML model summarized

All told we have ~400000 lines of patron circulation records. By creating a decision tree classifier based on three criteria: **total checkouts**, **total renewals**, and **circulation active year** we are going to attempt to be able to predict if a user is: **Adult**, **Young Adult**, or **other**.

We are going to look at the accuracy of our model and tweak some hyperparameters to improve that accuracy


### Let's start the code!

The next cell will built the lists that will hold our features and targets

In [ ]:
features = ["Total Checkouts", "Total Renewals", "Circulation Active Year"]
#The usual notation is to put all of the features into a variable called X
X = sf_data[features]

target = sf_data["Patron Type Definition"]
#Similarly the target is often put in a variable called y
y = target


With these pieces done we'll now build our ML Decision Tree Classifier. This might look complex but each line has some comments that explain what it is doing. One thing to note specifically is that line 13 does the work of creating 4 subsets of our data with the `train_test_split` function. Basically rows of training features with corresponding targets, then rows of testing features with the corresponding targets. We split our rows of data into a proportion set by the value `test_size`. Here we start with **0.2**. In other otherwords, we'll use 20% of our data to test our model and the remaining 80% to do the training.

In [ ]:
#what percentage of the dataset will we devote to testing our tree
#the remaining 80% of the data will be used for training
test_percent = 0.2


#train_test_split takes our data and creates for different sets with it

#A quick note here. Just like how we gave multiple values to
#the print statement as input, functions can return multiple values
#that is why we have this notation of four values seperated by commas
#on the left hand side of the = sign

X_train, X_test, y_train, y_test = train_test_split(X, \
                                                    y, \
                                                    test_size= test_percent)
# Create a Decision Tree classifer object
treeClass = DecisionTreeClassifier()

# We give our decision tree our training data. This is that it uses to build the tree
treeClass = treeClass.fit(X_train,y_train)

# We now test how accurate the tree is by giving it our testing data and seeing how
#the decision tree it created scores those values
y_pred = treeClass.predict(X_test)



## That's it?

Yup that's it. We separated the data into features and our target, then we split the rows in testing and training data. Line 17 above built the model in a variable called `treeClass`. Line 20 trained it, or in other parlance _fit_ it against the data. Line 24 tested against the test set, or in otherwords _predicted_ what guess our model would make. 

Let's evaluate the model by printing out how accurate it is. The next cell does this. There are many different measures you can apply against a model but we'll just use 'accuracy'. This is strictly how good the model can indentify true-positives in the data.

In [ ]:
#What is the accuracy of our ML Model?
metrics.accuracy_score(y_test,y_pred)

### Something to look out for!

Your target and features should be mutually exclusive. Or put in other words you shouldn't use the thing you are trying to guess in the evidence that you are using to build your model. If you get 1.0 everytime you test for accuracy there is something wrong with your choices for features and targets, you'll need to rethink how you distributed each column. This hint will help you when you get to the week 4 homework.

## Visualzing our tree


We can also 'print out our tree' to the screen to see what it looks like. It's is not as visual at the Titanic example above but with a bit of practice it should be easy to make sense of. The three below is a represenation of that Titanic example but drawn out in a way that matches how our code will look. (I've made a few changes here as you can probably tell, our models can only numbers and this diagram has text labels)

You read this version by going line by line left to right. the `|` symbols are there to help you see the continuation of questions.

![Ascii Tree Representation](https://raw.githubusercontent.com/elibtronic/lja_python_for_librarians/main/tree_ascii.png)

The cell below will print out the tree associated with our model. By tracing through the branches we can see what questions are the most important ones to the tree (ie. the ones it asks first) This is useful for us to make sense of what the model is doing, not just the results of it.

In [ ]:
printed_tree = export_text(treeClass,feature_names=features)
print(printed_tree)

So right out of the box the ML model can accurately classify **71%** of the data in the dataset. Let's just take a second to think about this. Based on just 3 values we can guess the type of patron we are looking at with 71% accuracy. Now imagine what services like Google can do with the data they have. It is worth noting though that the model is only as good as the data it is built on. Prejudiced data will lead to prejudiced models.

Another question to consider it: how high should accuracy be? That's a great question that varies based on the data set or the ML model used. One thing I can say is that value nevers get to 100% (not in any real data set that's for sure)

----

## Hyperparameters - Testing Set Size

Let's return to our code. We are now going to fiddle with some paramaters to hopefully increase accuracy. In our next cell we are going to experiment with using different amounts of our data for testing. As you'll remember we started with **20%** of our data for testing.  

We'll also graph out the accuracy when we adjust this value. We'll be using a Matplotlib [line graph](https://www.w3schools.com/python/matplotlib_plotting.asp) to accomplish this.

Run the next cell a few times to see your graph change shape. There is of calculations happening in this cell so it might take up to a minute to run.

In [ ]:
#A list of different values for testing sizes we'll loop through below
#values should be between 1 - 99, ie there must be at least 1% devoted to training
testing_percents = [1,5,10,20,30,99]

#used in our graph
accuracy = []
training_percents = []

for test_ratio in sorted(testing_percents):

    
    X_train, X_test, y_train, y_test = train_test_split(X, \
                                                        y, \
                                                        test_size=test_ratio/100.0)
    #Build/train/test our model
    treeClassTest = DecisionTreeClassifier()
    treeClassTest = treeClassTest.fit(X_train,y_train)
    y_pred = treeClassTest.predict(X_test)
    
    #Calculate our scores and add them to the list that will be used
    # by the graph
    score = metrics.accuracy_score(y_test,y_pred)
    accuracy.append(score)
    training_percents.append(100 - test_ratio)

    
plt.plot(training_percents,accuracy)
plt.ylabel("Accuracy in %")
plt.xlabel("Training Size %")
plt.show()

## Why is my graph changing?

If you've run the cell above a few times you'll of noticed that your graph will change shape. (Give it a try now if you haven't yet) So why is this the case? The creation of the ML model requires some initial random guessing to construct the tree. Everytime time you run it, the guess that is starts with is different and you get a different shape of graph. If we wanted to we could set something called a `random_seed` to get some repeatable results, but I've chosen not to do that so you can get a real sense of how random this process is. Wikipedia has a [quick description](https://en.wikipedia.org/wiki/Random_seed) if you'd like to know more.

Pay attention to the lowest and highest value on the Y-axis. (Accuracy in %) as you re-run the cell. In my testing varying the testing set size only gave me a difference of about 2-3%.


## Hyperparameters - Maximium Tree Depth

We saw in our initial example of the Titanic dataset that the tree was only 3 levels deep. As it turns out maximum tree depth is another hyperparameter we can set. The cell below goes through an investigation of this that is very similar to what we did with testing set size. Run it a few times to see that different graph being drawn.

You'll notice that in line 19 we add a parameter to the creation of the Decision Tree `max_depth`. This is what will restrict our tree size when we do our training/testing cycle. Every time we loop through our model build (line 10) we change `max_depth` to a different value to see how it effects our accuracy score.


In [ ]:
#We'll just keep our testing percent set to 20% so we can observe the changes
#setting a maximum depth has
test_percent = 20

max_options = [1,2,3,4,5,10,20,30]

accuracy = []
tree_max = []

for max_d in max_options:
    X_train, X_test, y_train, y_test = train_test_split(X, \
                                                        y, \
                                                        test_size=test_percent/100.0)
    
    
    #Build/train/test our model
    #We set maximum depth in the DecisionTreeClassifer when we first create the variable
    #with the paramater max_depth
    treeClassTest = DecisionTreeClassifier(max_depth=max_d)
    treeClassTest = treeClassTest.fit(X_train,y_train)
    y_pred = treeClassTest.predict(X_test)
    
    #Calculate our scores and add them to the list that will be used
    # by the graph
    score = metrics.accuracy_score(y_test,y_pred)
    accuracy.append(score)
    tree_max.append(max_d)

    
plt.plot(max_options,accuracy)
plt.ylabel("Accuracy in %")
plt.xlabel("Maximum Depth of Tree")
plt.show()

## Question 1

You've just adjusted training set size and maximum depth. Take a look at the y-axis for each graph (Accuracy in %). Which hyperparamater has a stronger affect on the accuracy?

The hyperparameter with the stongest effect is... 


I know this because...

## Tuning multiple Hyperparameters

Use the next cell to experiment with different values of both of those values to see if you can really tweak your model. You just need to change lines 2 & 4. Use this cell to answer questions 2 & 3.

In [ ]:
#Any value between 1 and 99
test_percent = 
#Any value between 1 and 10
max_d = 

X_train, X_test, y_train, y_test = train_test_split(X, \
                                                    y, \
                                                    test_size=test_percent/100.0)
#Build/train/test    
treeClassTest = DecisionTreeClassifier(max_depth=max_d)
treeClassTest = treeClassTest.fit(X_train,y_train)
y_pred = treeClassTest.predict(X_test)

#Accuracy?
print("Calculated accuracy: ")
print(metrics.accuracy_score(y_test,y_pred))


#Visualize tree
print("\nFinal tree found")
printed_tree = export_text(treeClass,feature_names=features)
print(printed_tree)

## Question 2

What combination of the 2 parameters created the highest accuracy?

The combination that created the hightest accuracy was...

## Question 3

What combination of the 2 paramester created the lowest accuracy?

The combination that created the lowest accuracy was...

## One last word on parameters...

Here's another takeaway from ML that I suggest you remember. You need to know the dynamics of the model you are using so that you can tune parameters and get the best accuracy possible. But what is a good accuracy? When do you stop adjusting parameters? How **good** can a ML model get generally? What happens if you use all of your data for training? 

These are all great questions. I don't know the answers... It all depends on the situation I guess. When you see AI and algorithms in the news I encourage you to think about these questions and how they should impact our understanding and use of ML.

----


# Making Predictions

Let's just take a brief digression into what else we can do with an ML model. We can also ask it to make predictions on hypothetical data we feed it. So in this case we give it pretend values for **total checkouts**, **total renewal**, and **circ year active** and it gives us a guess of what category that pretend datapoint would fall into.

First let's rebuild our model so we know we have the correct one loaded

In [ ]:
#what percentage of the dataset will we devote to testing our tree
#the remaining 80% of the data will be used for training
test_percent = 0.2


#train_test_split takes our data and creates for different sets with it
X_train, X_test, y_train, y_test = train_test_split(X, \
                                                    y, \
                                                    test_size= test_percent)
# Create a Decision Tree classifer object
treeClass = DecisionTreeClassifier()

# We give our decision tree our training data. This is that it uses to build the tree
treeClass = treeClass.fit(X_train,y_train)

# We now test how accurate the tree is by giving it our testing data and seeing how
#the decision tree it created scores those values
y_pred = treeClass.predict(X_test)

print("Accuracy?")
metrics.accuracy_score(y_test,y_pred)

Let's remind ourselves what categories this classifier can guess:

In [ ]:
sorted(sf_data["Patron Type Definition"].unique())


- category `1` which represents the **'Adults'** & **'Seniors'** from the original dataset 
- category `2` which represents the **'Young Adult'** users from the original dataset
- category `0` which represents the misc. categories of users from the original dataset

In the next cell change the values in the list `guess` to see what the model predicts. The comments are there to help describe what value you are modifying. You'll see the classification guessed by the model and the probability it has assigned to each of the three categories. (The one with the highest probability is the guess it will make)

You can also try to use values from the `sf_data.sample(10)` cell above to be able to verify that the  target predicted matches the actual target for that set of features that you set.

In [ ]:
guess = [
        100,   #Total Checkouts 
        10,    #Total Renewals
        2020   #Circ year active. (between 2000 and 2020 would be a good range)
        ]

dp = pd.DataFrame([guess],columns=X_test.columns)
print("Predicted Class: ")
print(treeClass.predict(dp))
print("----")
print("Probably of guess: ")
print(treeClass.predict_proba(dp))


----

# A Random Forest 

We'll level up and now look at Random Forest Classifiers. These, as the name implies, are ML models that generate a collection of trees and pick the best tree once training is done. With the random forest the new hyperparameter we need to adjust is called `n_estimators` which is the number of trees in our forest. Run the next cell to get a baseline on what we can get with a RandomForest.


In [ ]:
# We keep testing at 20 as we have been doing.
# This parameter has been involved in all of the models we have built 
test_percent = 20

X_train, X_test, y_train, y_test = train_test_split(X, \
                                                    y, \
                                                    test_size=test_percent/100.0)

#Create Random Classifier
clf = RandomForestClassifier(n_estimators=5)

#Train
clf.fit(X_train,y_train)

#Predict
y_pred = clf.predict(X_train)

#Accuracy
print('Accuracy ?')
metrics.accuracy_score(y_train,y_pred)

Wow, that's quite a jump. I got almost a 8% increase in accuracy by using a different ML model. (Your results will probably vary) 

Let's graph out a parametric sweep of of the `n_estimators` to see how that tweaks accuracy. (Heads up: this cell will take a while to run as well)

In [ ]:
test_percent = 20

n_options = [1,5,20,30,40,50]

accuracy = []
n_score = []

for n in n_options:
    X_train, X_test, y_train, y_test = train_test_split(X, \
                                                        y, \
                                                        test_size=test_percent/100.0)
    
    #Build/train/test
    clf = RandomForestClassifier(n_estimators=n)
    clf = clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    
    #Calculate accuracy for our graph
    score = metrics.accuracy_score(y_test,y_pred)
    accuracy.append(score)
    n_score.append(n)
        
plt.plot(n_score,accuracy)
plt.ylabel("Accuracy in %")
plt.xlabel("N Estimators")
plt.show()

Similarly here we see our graph changing shape each time you run the cell. You might even notice a difference between this cell and the one above. One of the wonders of random processes. You can keep repeating them and keep getting different values!


Use the cell below to answer questions 4 & 5. You'll need change lines 2 & 4

**Quick note**: Visualizing a forest is a bit trickier than visualizing a single tree. For the sake on simplicity our code snippet here will print the first tree found in the forest. (It might not be the best performing tree but at least we get a sense of the important features looked at).

In [ ]:
#pick values between 1 and 99
test_percent = 0

#pick values betwen 5 and 50
number_estimators = 0

X_train, X_test, y_train, y_test = train_test_split(X, \
                                                    y, \
                                                    test_size=test_percent/100.0)

#Create Random Forest Classifier
clf = RandomForestClassifier(n_estimators=number_estimators)

#Train
clf.fit(X_train,y_train)

#Predict
y_pred = clf.predict(X_train)

#Accuracy
print('Accuracy ?')
print(metrics.accuracy_score(y_train,y_pred))

#Visualize the first tree in this forest
print("\nFirst tree in this forest")
printed_tree = export_text(clf.estimators_[0],feature_names = features)
print(printed_tree)

## Question 4

What combination of `test_percent` and `number_estimators` gets you the highest accuracy?

I got the highest accuracy with...

## Question 5

What combination of `test_percent` and `number_estimators` gets you the lowest accuracy?

I got the lowest accuracy with...

## Congrats!

You've completed your investigation of ML models: how to build them, how to train/test them and finally how to adjust parameters. Couple of thing before you attempt the homework. I've simplified our investigation of parameters, there is *a lot* more tweaks we can make. Check out the webpage for the [Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) for a brief glimpse of all the options you can tweak. 

A lesson which I think rings true (and I hope you agree with me) is that ML models are black boxes. We add in data (hopefully without any exploitive, discriminatory components) turn some knobs and dials and get a result. This is fine when you want to make some innocuous insights into a spreadsheet of data that you have, this is a bit worrisome when you're building an ML model that will decide important parts of a person's life. 

Please head over to the **Homework Notebook** to try some questions on these topics.